In [4]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

# Define the hyperparameters
HP_ARCHITECTURE_1 = hp.HParam('architecture_1', hp.Discrete([5, 39, 40, 31, 42, 1]))
HP_ARCHITECTURE_2 = hp.HParam('architecture_2', hp.Discrete([5, 61, 51, 28, 39, 26, 2120, 14, 1]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.01, 0.009, 0.008, 0.007, 0.006, 0.005, 0.004, 0.003, 0.002, 0.001]))

METRIC_MSLE = 'msle'

# Configure the hyperparameters and metrics for TensorBoard
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_ARCHITECTURE_1, HP_ARCHITECTURE_2, HP_LEARNING_RATE],
        metrics=[hp.Metric(METRIC_MSLE, display_name='MSLE')],
    )


In [ ]:
#ou alors

"""

# Define your hyperparameters
learning_rate = 0.001
units = 64

# Create a summary writer and start a new session
writer = tf.summary.create_file_writer('logs/hparams')

# Define the hyperparameter configuration
hyperparams = {
    'learning_rate': learning_rate,
    'units': units
}

# Log the hyperparameters
with writer.as_default():
    hp.hparams(hyperparams)  # Log the hyperparameters

"""
# Start the TensorBoard server to visualize the logs
# Open a terminal or command prompt and navigate to the directory containing your code
# Run the fol